In [1]:
import sys

sys.path.append("../")

import os

from dotenv import load_dotenv

from src.train import train_and_save_model

# load environment variables
load_dotenv()

path = os.getenv("DATA_FOLDER")

2025-08-26 13:59:13.796418: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-26 13:59:13.948489: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756209554.015484   58773 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756209554.033955   58773 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756209554.175524   58773 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Model training

In [2]:
model = train_and_save_model(path=path)

I0000 00:00:1756209562.493396   58773 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13499 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4080 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


Epoch 1/6
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 25s 10ms/step - accuracy: 0.6971 - loss: 0.7102 - val_accuracy: 0.8560 - val_loss: 0.4019
Epoch 2/6
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 24s 10ms/step - accuracy: 0.8459 - loss: 0.3992 - val_accuracy: 0.9140 - val_loss: 0.2290
Epoch 3/6
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.8943 - loss: 0.2718 - val_accuracy: 0.9350 - val_loss: 0.1806
Epoch 4/6
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 23s 10ms/step - accuracy: 0.9130 - loss: 0.2170 - val_accuracy: 0.9450 - val_loss: 0.1669
Epoch 5/6
2334/2334 ━━━━━━━━━━━━━━━━━━━━ 24s 10ms/step - accuracy: 0.9251 - loss: 0.1830 - val_accuracy: 0.9410 - val_loss: 0.1660


# Test model loading and prediction 

In [3]:
from tensorflow.keras.models import load_model  # type: ignore

In [4]:
model = load_model("./../models/full_model_1D.keras")

In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, 64)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 64, 64)         │     2,725,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,190,923 (31.25 MB)

 Trainable params: 2,730,307 (10.42 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,460,616 (20.83 MB)

## Mock input as JSON

In [6]:
import json

import pandas as pd

names = ["tweet_id", "entity", "sentiment", "text"]
val_df = pd.read_csv(filepath_or_buffer=path + "/twitter_validation.csv", names=names)

In [7]:
idx_to_label = {
    0: "Negative",
    1: "Neutral",  # or "Irrelevant" — choose one
    2: "Positive",
}

In [20]:
# Select row 69 (remember: Python uses 0-based indexing)
row = val_df.iloc[14]  # row 69 in human terms
# Create JSON with "entity" and "text"
data = {"entity": row["entity"], "text": row["text"]}
request = json.dumps(data, indent=2)
print("Entity & Text JSON:")
print(request)
print(f"\nSentiment: {row['sentiment']}")

Entity & Text JSON:
{
  "entity": "NBA2K",
  "text": "@NBA2K game sucks... down by 2 with 38 seconds left and my team intentionally fouls"
}

Sentiment: Negative


## Format and predict

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import convert_to_tensor

from src.formatting import format_input

In [21]:
request_parsed = json.loads(request)
formatted_text = format_input(request_parsed["entity"], request_parsed["text"])

In [22]:
raw_text_data = convert_to_tensor([formatted_text], dtype=tf.string)
print(raw_text_data)
print(raw_text_data.shape)
prediction = model.predict(raw_text_data)
predicted_class_idx = int(np.argmax(prediction, axis=1)[0])

predicted_label = idx_to_label[predicted_class_idx]
confidence = float(np.max(prediction))
print("predicted_class: ", predicted_label, "confidence: ", confidence)

tf.Tensor([b'What is the sentiment about [ENTITY] <e>NBA2K</e> in: @NBA2K game sucks... down by 2 with 38 seconds left and my team intentionally fouls'], shape=(1,), dtype=string)
(1,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
predicted_class:  Negative confidence:  0.999988317489624
